<img src='https://gitlab.eumetsat.int/eumetlab/oceans/ocean-training/tools/frameworks/-/raw/main/img/OSI-SAF_banner.png' align='right' width='100%'/>

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1a_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb" target="_blank"><< Accessing Metop SST metagranule data through IFREMER opensearch queries</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_2_OSI_SAF_SST_file_structure.ipynb" target="_blank">Exploring OSI SAF SST product structure >></a>

<font color="#138D75">**EUMETSAT OSI SAF Training Service**</font> <br>
**Copyright:** 2023 EUMETSAT <br>
**License:** MIT

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://mybinder.org/v2/git/https%3A%2F%2Fgitlab.eumetsat.int%2Feumetlab%2Foceans%2Focean-training%2Fsensors%2Flearn-osi-saf-sst/HEAD?urlpath=%2Ftree%2F1_OSI_SAF_sst_introductory%2F1_1b_OSI_SAF_SST_data_access_IFREMER_FTP.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h3>Learn OSI SAF sea surface temperature: Introductory</h3></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
This notebook has the following prerequisites:
- **<a href="https://osi-saf.eumetsat.int/register" target="_blank">A EUMETSAT OSI SAF account</a>** if you are using or plan to use the EUMETSAT OSI SAF data.

There are no prerequisite notebooks for this module.
</div>
<hr>

# 1.1b Accessing OSI SAF SST products through the IFREMER FTP server
### Data used 

| Product Description | Product Navigator | OSI SAF website link | OSI SAF Identifier |
|:--------------------:|:-------------:|:-----------------:|:-----------------:|
| Global Metop Sea Surface Temperature | EO:EUM:DAT:METOP:GLB-SST-NC | <a href="https://osi-saf.eumetsat.int/products/osi-201-b" target="_blank">link</a>| OSI-201-b | 
| Meteosat Sea Surface Temperature | EO:EUM:DAT:METEOSAT:OSIHSST-NC | <a href="https://osi-saf.eumetsat.int/products/osi-206-a" target="_blank">link</a>| OSI-206-a | 


### Learning outcomes

At the end of this notebook you will know;
* How to download different OSI SAF sea surface temperature (SST) products using FTP access

### Outline

Data from OSI SAF is available through multiple sources. Here we will guide you through FTP access to OSI SAF SST Data.

<div class="alert alert-info" role="alert">

## <a id='TOC-TOP'></a>Contents

</div>
    
 1. [Creating our workspace](#section1)
 1. [Setting login information](#section2)
 1. [Downloading via FTP requests](#section3)

<hr>

<div class="alert alert-info" role="alert">

## <a id='section1'></a>1. Creating the workspace
[Back to top](#TOC-TOP)

</div>

We begin by importing all of the libraries that we need to run this notebook. If you have built your python using the environment file provided in this repository, then you should have everything you need. For more information on building environment, please see the repository **<a href="../README.md" target="_blank">README</a>**.

In [1]:
# library imports 
import os           # a library that allows to access to basic operating system commands like making directories
import json         # a library that helps with JSON format files
import ftplib       # a library that supports FTP protocol use
import warnings     # a library that controls python warnings
warnings.filterwarnings('ignore')

Next we will create a download directory to store the products we will download in this notebook.

In [2]:
download_dir = os.path.join(os.getcwd(), "products")
os.makedirs(download_dir, exist_ok=True)

<div class="alert alert-info" role="alert">

## <a id='section2'></a>2. Setting login information
[Back to top](#TOC-TOP)

</div>

We will access OSI SAF data from the OSI SAF Lower-Mid-Latitude Centre FTP server.

In order to allow us to download data, we need to provide our credentials. We can do this in two ways; either by creating a file called `.eumetsat_osi_saf_sst_credentials` in our home directory (*option 1 - recommended*) or by supplying our credentials directly in this script (*option 2*). 

#### Option 1: creating  `.eumetsat_osi_saf_sst_credentials` in our home directory

For most computer systems the home directory can be found at the path \user\username, /users/username, or /home/username depending on your operating system.

In this file we need to add the following information exactly as follows;

```
{
"username": "<provided_username>",
"password": "<provided_password>"
}
```

You must replace `<provided_username>` and `<provided_password>` with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in Sea Ice data. 

Once you have created an <a href="https://osi-saf.eumetsat.int/register" target="_blank">EUMETSAT OSI SAF account</a>, the generic password and credentials to access OSI SAF Lower-Mid-Latitude Centre server should have been sent to you via email.

Make sure to save the file without any kind of extension.

Once you have done this, you can read in your credentials using the commands in the following cell. These will be used to generate a time-limited token, which will refresh itself when it expires.

Reading credential information

In [3]:
# read credentials
with open(os.path.join(os.path.expanduser("~"),'.eumetsat_osi_saf_sst_credentials')) as json_file:
    credentials = json.load(json_file)

Loading credentials

In [4]:
username = credentials['username'] 
password = credentials['password']

Option 2: provide credentials directly
You can provide your credentials directly as follows; by uncommenting the following lines.

In [5]:
#username = "<provided_username>"
#password = "<provided_password>"

You must replace <provided_username> and <provided_password> with the information provided to you after you have registered to the OSI SAF website and mentionned your interest in SST data.

Once you have created an EUMETSAT OSI SAF account, the generic password and credentials to access OSI SAF Lower-Mid-Latitude  server should have been sent to you via email.

Note: this method is convenient in the short term, but is not really recommended as you have to put your provided username and password in this notebook, and run the risk of accidentally sharing them. This method also requires you to authenticate on a notebook-by-notebook basis.

<div class="alert alert-danger" role="alert">

## <a id='section3'></a>3. Launch FTP requests
[Back to top](#TOC-TOP)

</div>

### Global Metop Sea Surface Temperature :

Let's start by creating a specific folder to hold the Global Metop Sea Surface products

In [6]:
download_dir = os.path.join(os.getcwd(), "products/global")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [7]:
path = '/cersat-rt/project/osi-saf/data/sst/l3c/global/glob_avhrr_metop_b/2022/244/'
filename = '20220901000000-OSISAF-L3C_GHRSST-SSTsubskin-AVHRR_SST_METOP_B_GLB-sstglb_metop01_20220901_000000-v02.0-fv01.0.nc'

In [8]:
# connect ftp and download file
ftp = ftplib.FTP("eftp1.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/global/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

### Meteosat Sea Surface Temperature :

Let's start by creating a specific folder to hold the Meteosat Sea Surface Temperature products

In [9]:
download_dir = os.path.join(os.getcwd(), "products/meteosat")
os.makedirs(download_dir, exist_ok=True)

Set the download path

In [10]:
path = '/cersat-rt/project/osi-saf/data/sst/l3c/east_atlantic_west_indian/meteosat/2022/244'
filename = '20220901120000-OSISAF-L3C_GHRSST-SSTsubskin-SEVIRI_SST-ssteqc_meteosat11_20220901_120000-v02.0-fv01.0.nc'

In [11]:
# connect ftp and download file
ftp = ftplib.FTP("eftp1.ifremer.fr") 
ftp.login(username, password)
ftp.cwd(path)
ftp.retrbinary("RETR " + filename, open('products/meteosat/' + filename, 'wb').write)
ftp.quit()

'221 Goodbye.'

<a href="../Index.ipynb" target="_blank"><< Index</a>
<br>
<a href="./1_1a_OSI_SAF_SST_data_access_IFREMER_Opensearch.ipynb" target="_blank"><< Accessing Metop SST metagranule data through IFREMER opensearch queries</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_2_OSI_SAF_SST_file_structure.ipynb" target="_blank">Exploring OSI SAF SST product structure >></a>
<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>